# Projeto Covid19
## seguindo digital inovation one

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
url = 'https://github.com/neylsoncrepalde/projeto_eda_covid/blob/master/covid_19_data.csv?raw=true'

In [ ]:
df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])
df

In [ ]:
import re

def corrigi_colunas(col_name):
    return re.sub(r"[/| ]", "", col_name.lower())

In [ ]:
df.columns = [corrigi_colunas(col) for col in df.columns]
df

In [ ]:
df_brasil = df.loc[df.countryregion =='Brazil'].reset_index()
df_brasil

In [ ]:
px.line(df_brasil, 'observationdate','confirmed', title='casos confirmados no brasil')

# novos casos por dia

In [ ]:
df_brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else df_brasil['confirmed'].iloc[x] - df_brasil['confirmed'].iloc[x-1],
    np.arange(df_brasil.shape[0])
))

In [ ]:
px.line(df_brasil, x='observationdate', y= 'novoscasos', title='novos casos por dia)

# Mortes

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df_brasil.observationdate, y=df_brasil.deaths, name = 'Mortes',
              mode='lines+markers', line={'color':'red'})
)

fig.update_layout(title="Mortes por Covid 19  no brasil")

fig.show()

# Taxa de Crescimento

In [ ]:
def taxa_crescimento(data, variable, data_inicio=None, data_fim=None):
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[variable]>0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
    
    if data_fim==None:
        data_fim = data.observationdate.iloc[-1]
    else:
        data_fim = pd.to_datetime(data_fim)
        
    passado =  data.loc[data.observationdate == data_inicio, variable].values[0]
    presente = data.loc[data.observationdate == data_fim, variable].values[0]
    
    n = (data_fim-data_inicio).days
    
    taxa=(presente/passado)**(1/n) - 1
    
    return taxa*100
    

In [ ]:
taxa_crescimento(df_brasil, 'confirmed')

In [ ]:
def taxa_crescimento_diaria(data, variable, data_inicio = None):
    if data_inicio==None:
        data_inicio = data.observationdate.loc[data[variable] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
    
    data_fim= data.observationdate.max()
    
    n = (data_fim-data_inicio).days
    
    taxas = list(map(
        lambda x:  (data[variable].iloc[x] - data[variable].iloc[x-1]) / data[variable].iloc[x-1] ,
        range(1, n+1)
    ))
    
    return np.array(taxas) * 100

In [ ]:
tx_dia = taxa_crescimento_diaria(df_brasil,'confirmed')

In [ ]:
tx_dia

In [ ]:
df_brasil['confirmed']

In [ ]:
df_brasil = df_brasil.loc[df_brasil['confirmed'] !=0]

In [ ]:
primeiro_dia = df_brasil.observationdate.loc[df_brasil.confirmed >0].min()
px.line(x=pd.date_range(primeiro_dia, df_brasil.observationdate.max())[1:],
       y=tx_dia, title="taxa de crescimendo diario no Brasil")

# predições 

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt


In [ ]:
confirmados = df_brasil.confirmed
confirmados.index =df_brasil.observationdate
confirmados

In [ ]:
res= seasonal_decompose(confirmados)
dig,(ax1, ax2, ax3, ax4 ) = plt.subplots(4,1,figsize=(10,8))

ax1.plot(res.observed)
ax2.plot(res.trend)
ax3.plot(res.seasonal)
ax4.plot(confirmados.index, res.resid)
ax4.axhline(0,linestyle='dashed', c='black')
plt.show()

# Arima

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima

modelo = auto_arima(confirmados)

In [ ]:
fig = go.Figure(go.Scatter(
    x=confirmados.index, y=confirmados, name="observados"
))

fig.add_trace(go.Scatter(
    x=confirmados.index, y=modelo.predict_in_sample(), name="preditos"
))

fig.add_trace(go.Scatter(
    x=pd.date_range('2020-05-20','2020-06-20'), y= modelo.predict(31), name ='forecast'
))

fig.update_layout(title = "previsão de casos para os proximos 30 dias")
fig.show()

# Modelo de crescimento

In [ ]:
!conda install -c conda-forge fbprophet -y

In [ ]:
from fbprophet import Prophet

In [ ]:
train = confirmados.reset_index()[:-5]
test = confirmados.reset_index()[-5:]

train.rename(columns = {'observationdate':'ds', 'confirmed':'y',inplace = True})
test.rename(columns = {'observationdate':'ds', 'confirmed':'y',inplace = True})

profeta = Prophet(growth='logistic', changepoints=['2020-03-21','2020-03-30','2020-04-25','2020-05-03','2020-05-10'])



pop = 21146356
train['cap'] = pop

profeta.fit(train)

future_dates =profeta.make_future_dataframe(peirods =200 )
future_dates['cap'] =pop
forecast = profeta.predict(future_dates)

In [115]:
!pip install pystan

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x= forecast.ds, y=forecast.yhat,name="predição"))
fig.add_trace(go.Scatter(x= train.ds, y=train.y,name="observados - treino"))
fig.update_layout(title = "predição de casos confirmados no brasil")
dig.show